# Notes

## Oct 14 Notes
Attempted to figure out why beam search is so slow.
- I noticed that some of the outputs of both beamsearch and hillclimbing have obvious redundancies, i.e. CX(0,1) SWAP(0,1), that could be simplified. I bet this would improve our performance relative to Qiskit.
- Beam search does not strictly improve with increasing beam_width, no guarantee on LGF heuristic
- Method can do really good for log-linear scaling, 6 qubits, even for uniformly generated Cliffords
- profiling exercise revealed that the update beam step is what's taking so long, and that this can be made fast by not checking for previously visited nodes
    - however, doing so compelely ruins performance, we rarely get a solution
    - further investigation reveals that in these cases, the algorithm gets stuck. THe reason for thsi is that the Pauli gates have identity tableau matrices once the phase bits are dropped,
    so they don't change the state at all.
    - For this reason I think we should drop X, Y, Z from our gate set, and probably also SWAP - these were added in for compatibility with Qiskit
    - However, might still have an issue that X, Y, Z can be built out of H, S, and so we might just be obscuring, rather than addressing, the problem

In [1]:
import tqdm
from typing import Dict
import numpy as np
import torch
import torch.nn as nn
import rubiks.clifford as cl
import rubiks.lgf as lgf
from qiskit.quantum_info import Clifford, random_clifford
from qiskit.synthesis import synth_clifford_full
import cProfile

In [10]:
num_qubits = 6

SEED = 123
use_qiskit = False
device = torch.device('cpu')
drop_phase_bits = True
scaling = 'log-linear'
#scaling = 'linear'
data_dir = f"data/data_n_{num_qubits}_drop_phase_bits_scaling_{scaling}/"
high = cl.max_random_sequence_length(num_qubits, scaling)
print(f"for scaling={scaling}, high={high}")

lgf_model = lgf.LGFModel(
    num_qubits=num_qubits,
    device=device,
    rng=np.random.default_rng(SEED),
    hidden_layers=[32, 16, 4, 2],
    drop_phase_bits=drop_phase_bits,
    use_qiskit=use_qiskit,
).to(device)

lgf_model.load_state_dict(torch.load(data_dir + "checkpoint"))

for scaling=log-linear, high=155


<All keys matched successfully>

- Compare for different values of beam_width
    - should agree with previous results for beam_width=1
    - should be better for larger beam_widths
- Paper revisions
    - add beam search algorithm
    - update reuslts
        - consider using the win-rate
        - consider not going to very high max_iters

In [11]:
WEIGHT_DICT = {
    "cx": 1,
    "swap": 3,
}

In [12]:
sampling_method = 'random_walk'

if sampling_method == 'random_walk':
    sequence_length = high
    print("Sampling a Clifford via random walk")
    initial_sequence = cl.random_sequence(np.random.default_rng(), seq_length=sequence_length, num_qubits=num_qubits)
    initial_state = cl.sequence_to_tableau(initial_sequence, num_qubits)
else:
    print("Sampling a Clifford uniformly")
    initial_state = random_clifford(num_qubits=num_qubits)

Sampling a Clifford via random walk


In [13]:
results_dict = lgf.beam_search(initial_state=initial_state, lgf_model=lgf_model, beam_width=1, max_iter=100, drop_phase_bits=True, check=True)

count = results_dict['count']
success = results_dict['success']

if success is False:
    print(f"No solution found after {count} steps")
else:
    print(f"Solution found after {count} steps")
    print(f"Weighted distance to identity {cl.weighted_distance_to_identity(results_dict['move_history'], weight_dict=WEIGHT_DICT)}")
    print(results_dict['move_history'])

Time taken to build neighbor list of beams: 0.01486968994140625
Time taken to check for solution: 0.0006518363952636719
Time taken to compute LGF values: 0.0006566047668457031
Number of neighbors to sort through: 81
Time taken to update beam: 0.014231443405151367
Time taken to build neighbor list of beams: 0.01321268081665039
Time taken to check for solution: 0.0005507469177246094
Time taken to compute LGF values: 0.0003654956817626953
Number of neighbors to sort through: 81
Time taken to update beam: 0.011752843856811523
Time taken to build neighbor list of beams: 0.0072252750396728516
Time taken to check for solution: 0.0003235340118408203
Time taken to compute LGF values: 0.00022268295288085938
Number of neighbors to sort through: 81
Time taken to update beam: 0.00736236572265625
Time taken to build neighbor list of beams: 0.007241725921630859
Time taken to check for solution: 0.0003139972686767578
Time taken to compute LGF values: 0.00022077560424804688
Number of neighbors to sort 

In [ ]:
import hashlib
hashlib.sha1('hello'.encode("utf-8")).hexdigest()

In [ ]:
len(cl.Problem(num_qubits=num_qubits, initial_state=initial_state).move_set)

In [15]:
cl.Problem(num_qubits=num_qubits, initial_state=initial_state).move_set[5]

(0, 'z')

In [32]:
problem = cl.Problem(num_qubits=num_qubits, initial_state=initial_state)

for i_move, move in enumerate(problem.move_set):
    if np.array_equal(
        problem.move_set_array[i_move][:,:-1], 
        np.eye(N=2*num_qubits)
        ):
        print(i_move, move)

3 (0, 'x')
4 (0, 'y')
5 (0, 'z')
24 (1, 'x')
25 (1, 'y')
26 (1, 'z')
42 (2, 'x')
43 (2, 'y')
44 (2, 'z')
57 (3, 'x')
58 (3, 'y')
59 (3, 'z')
69 (4, 'x')
70 (4, 'y')
71 (4, 'z')
78 (5, 'x')
79 (5, 'y')
80 (5, 'z')


In [24]:
cl.Problem(num_qubits=num_qubits, initial_state=initial_state).move_set[4]

(0, 'y')

In [ ]:
results_dict = lgf.hillclimbing(initial_state=initial_state, lgf_model=lgf_model, max_iter=1000)

count = len(results_dict['move_history'])
success = results_dict['success']

if success is False:
    print(f"No solution found after {count} steps")
else:
    print(f"Solution found after {count} steps")
    print(f"Weighted distance to identity {cl.weighted_distance_to_identity(results_dict['move_history'], weight_dict=WEIGHT_DICT)}")
    print(results_dict['move_history'])

In [ ]:
print(f"Weighted distance to identity {cl.weighted_distance_to_identity(synth_clifford_full(initial_state), weight_dict=WEIGHT_DICT)}")

In [ ]:
state = initial_state @ cl.sequence_to_tableau(results_dict['move_history'], num_qubits=num_qubits)
np.array_equal(state.tableau[:,:-1], cl.sequence_to_tableau([], num_qubits=num_qubits).tableau[:,:-1])

In [ ]:
results_dict_hillclimbing = lgf.hillclimbing(initial_state=initial_state, lgf_model=lgf_model, max_iter=1000)
if results_dict_hillclimbing['success']:
    print(f"found solution in {len(results_dict_hillclimbing['move_history'])} steps")
    print(results_dict_hillclimbing['move_history'])

In [ ]:
state = initial_state @ cl.sequence_to_tableau(results_dict_hillclimbing['move_history'], num_qubits=num_qubits)
np.array_equal(state.tableau[:,:-1], cl.sequence_to_tableau([], num_qubits=num_qubits).tableau[:,:-1])

In [ ]:
# get the move history to be in the same format
state = initial_state @ cl.sequence_to_tableau(results_dict_hillclimbing['move_history'][::-1], num_qubits=num_qubits)
state == cl.sequence_to_tableau([], num_qubits=num_qubits)

In [ ]:
cProfile.run('lgf.beam_search(initial_state=initial_state, lgf_model=lgf_model, beam_width=5, max_iter=1000)')